# Fully Connected Tokyo 2025 Karakuri Workshop

In [1]:
import logging

# Colabではgit cloneする
import sys
sys.path.append("..")

logger = logging.getLogger(__name__)

TODO: Colabでの実行も想定し、.envを読まずともベタ書きも出来るような書き方にする。

In [2]:
import os
from src.config import Config

config = Config(_env_file="../.env")
os.environ["OPENAI_API_KEY"] = config.openai_api_key.get_secret_value()

In [3]:
from src.chat import setup_weave

setup_weave(
    wandb_api_key=config.wandb_api_key.get_secret_value(),
    weave_project_name=config.weave_project_name,
)

weave: Logged in as Weights & Biases user: k-obinata-trials.
weave: View Weave data at https://wandb.ai/k-obinata-trials-obinata-private/weave-quickstart/weave


# チャット機能のお試し

In [4]:
from src.chat import chat

response = chat(messages=[{"role": "user", "content": "プランを変更したいです"}])
print(response)

weave: 🍩 https://wandb.ai/k-obinata-trials-obinata-private/weave-quickstart/r/call/0199fdb1-f68b-7240-89aa-a85aca86c6b3


プランの変更をご希望の場合は、管理画面の「プラン設定」からいつでも変更が可能です。変更は即座に反映されますので、ご都合の良いタイミングでお手続きください。

ご不明な点がありましたら、詳しい操作方法もご案内いたしますので、お気軽にお知らせください。


weave: 🍩 https://wandb.ai/k-obinata-trials-obinata-private/weave-quickstart/r/call/0199fdb3-b2d0-737f-b992-434a7384d12a
weave: 🍩 https://wandb.ai/k-obinata-trials-obinata-private/weave-quickstart/r/call/0199fdb3-d1b6-793c-9403-db244fb39ebd
weave: 🍩 https://wandb.ai/k-obinata-trials-obinata-private/weave-quickstart/r/call/0199fdb4-416d-7b80-b9d9-77f1b61ece07


# 精度評価

In [5]:
from src.run_evaluation import (
    load_evaluation_dataset,
    QAAccuracyScorer,
    ContextHallucinationScorer,
    CommonSenseHallucinationScorer,
    ChatBotModel,
)
from weave import Dataset, Evaluation, Table

logger.info("Loading evaluation dataset...")
dataset = load_evaluation_dataset(file_path="../data/eval-onboarding.csv")
logger.info("Loaded %d evaluation examples", len(dataset))

qa_accuracy_scorer = QAAccuracyScorer()
context_scorer = ContextHallucinationScorer()
commonsense_scorer = CommonSenseHallucinationScorer()
model = ChatBotModel()

# Create evaluation
table = Table(dataset)
weave_dataset = Dataset(name="chatbot_evaluation", rows=table)
evaluation = Evaluation(
    dataset=weave_dataset,
    scorers=[qa_accuracy_scorer, context_scorer, commonsense_scorer],
)

logger.info("Starting evaluation...")
try:
    results = await evaluation.evaluate(model)
    logger.info("Evaluation completed successfully")
    logger.info("Results: %s", results)
except Exception:
    logger.exception("Evaluation failed")
    raise

weave: Evaluated 1 of 10 examples
weave: 🍩 https://wandb.ai/k-obinata-trials-obinata-private/weave-quickstart/r/call/0199fdb2-ccd5-7b09-a25a-1105972fab4d
weave: Evaluated 2 of 10 examples
weave: Evaluated 3 of 10 examples
weave: Evaluated 4 of 10 examples
weave: Evaluated 5 of 10 examples
weave: Evaluated 6 of 10 examples
weave: Evaluated 7 of 10 examples
weave: Evaluated 8 of 10 examples
weave: Evaluated 9 of 10 examples
weave: Evaluated 10 of 10 examples
weave: Evaluation summary {
weave:   "QAAccuracyScorer": {
weave:     "accuracy": {
weave:       "answer": {
weave:         "mean": 1.1
weave:       }
weave:     }
weave:   },
weave:   "ContextHallucinationScorer": null,
weave:   "CommonSenseHallucinationScorer": null,
weave:   "model_latency": {
weave:     "mean": 4.0956339836120605
weave:   }
weave: }


# プロンプト改善

In [6]:
from src.prompts import SYSTEM_PROMPT
from src.prompt_improvement import improve_prompt_from_feedback



result = improve_prompt_from_feedback(SYSTEM_PROMPT, feedback_csv_path="../data/feedback-logs.csv")
print("=== 改善後のプロンプト ===")
print(result.improved_prompt)
print("=== 変更理由 ===")
print(result.change_reason)
print("=== フィードバック分析 ===")
print(result.feedback_analysis)

=== 改善後のプロンプト ===
## 基本情報
あなたはカスタマーサポート向けSaaS『カラクリCSデジタルシリーズ』のカスタマーサポート担当です。
丁寧かつ迅速な対応を心掛けてください。

## 回答ルール
1. 回答前に必ずFAQを参照し、FAQやドキュメントに記載された内容を優先して案内してください。
2. FAQやドキュメントに情報が無い場合は、
   - 追加のヒアリングや詳細説明を行わず、速やかに『お問い合わせフォーム（https://example.com/inquiry）』のご案内に切り替えてください。
   - 「詳細なご案内には担当部署での確認が必要です。お手数ですが、以下のお問い合わせフォームよりご連絡ください。」など、期待を煽らず丁寧に案内してください。
3. サービスに直接関係しない一般的な質問（例：APIとは何か等）には、
   - 必要以上に一般的な説明や深堀りは行わず、「ご質問がカラクリCSデジタルシリーズのご利用に関する内容でしたら、具体的にお知らせください。」など、サービス関連のご質問が無いかを確認してください。
4. サービスに関する質問や改善要望には、FAQやドキュメントの情報をもとに具体的かつ丁寧に回答し、必要に応じてユーザーのニーズを深堀りしてください。

## 注意事項
- 不明点やFAQに無い内容については、無理に推測や期待を持たせる回答は避けてください。
- すべての回答は、ユーザーの時間を尊重し、簡潔かつ明確に行ってください。

=== 変更理由 ===
フィードバック分析結果から、主な不満点は『サービス非関連の質問への過剰な説明』と『FAQに情報が無い場合の不適切な深堀り・期待を煽る対応』であることが判明しました。これらを解決するため、
- FAQに無い場合は追加ヒアリングや詳細説明をせず、速やかに問い合わせフォーム案内へ切り替える明確な指示を追加。
- サービスに直接関係しない一般的な質問には、必要以上に説明せずサービス関連の質問が無いか確認するルールを明記。
- サービス関連の質問には従来通り丁寧な深堀りを推奨。
これにより、ユーザーの無駄な期待や混乱を防ぎ、迅速かつ的確な案内でサービス品質とユーザー体験の向上が期待できます。
=== フィードバック分析 ===
{'total_logs': 4, 'ne

# 改善示唆出し

In [7]:
from weave import Dataset, Evaluation, Table

from src.run_improvement import load_questions, ChatBotModel, KnowledgeGapScorer

logger.info("Loading questions...")
questions = load_questions(file_path="../data/eval-onboarding.csv")
logger.info("Loaded %d examples", len(questions))

model = ChatBotModel()
scorer = KnowledgeGapScorer()

table = Table([{"question": q["question"], "id": q["id"]} for q in questions])
dataset = Dataset(name="knowledge_improvement", rows=table)

evaluation = Evaluation(dataset=dataset, scorers=[scorer])

logger.info("Starting improvement evaluation...")
results = await evaluation.evaluate(model)
logger.info("Improvement evaluation finished: %s", results)

weave: Evaluated 1 of 10 examples
weave: Evaluated 2 of 10 examples
weave: Evaluated 3 of 10 examples
weave: Evaluated 4 of 10 examples
weave: Evaluated 5 of 10 examples
weave: Evaluated 6 of 10 examples
weave: Evaluated 7 of 10 examples
weave: Evaluated 8 of 10 examples
weave: Evaluated 9 of 10 examples
weave: Evaluated 10 of 10 examples
weave: Evaluation summary {
weave:   "KnowledgeGapScorer": {
weave:     "knowledge_gap": {
weave:       "flag": {
weave:         "mean": 0.6
weave:       },
weave:       "raw": {
weave:         "lack_of_knowledge": {
weave:           "mean": 0.6
weave:         },
weave:         "lack_of_understanding": {
weave:           "mean": 0.4
weave:         }
weave:       }
weave:     }
weave:   },
weave:   "model_latency": {
weave:     "mean": 3.048652458190918
weave:   }
weave: }
